In [79]:

import pandas as pd
import numpy as np


%matplotlib inline

In [80]:
#Bring in the data

nba = pd.read_csv('./data/nba_combined.csv')

In [81]:
nba.columns

Index(['index', 'dataset', 'date', 'teams', 'venue', 'rest_days',
       'starting_lineups', 'unnamed:_35', 'unnamed:_36', 'unnamed:_37',
       'unnamed:_38', 'main_referee', 'crew_referees', 'opening_odds',
       'spread', 'total', 'movements', 'closing', 'moneyline', 'index.1',
       'away_team', 'away_rest', 'starting_lineups.1', 'home_starter2',
       'home_starter3', 'home_starter4', 'home_starter5', 'ref_3', 'away_line',
       'away_total', 'total_moves', 'closing total', 'away_moneyline',
       'mov_5_1q', 'mov_5_2q', 'mov_5_3q', 'mov_5_4q', 'mov_5_ot1',
       'mov_5_ot2', 'mov_5_ot3', 'mov_5_ot4', 'mov_5_f', 'mov_5_min',
       'mov_5_fg', 'mov_5_fga', 'mov_5_3p', 'mov_5_3pa', 'mov_5_ft',
       'mov_5_fta', 'mov_5_or', 'mov_5_dr', 'mov_5_tot', 'mov_5_a', 'mov_5_pf',
       'mov_5_st', 'mov_5_to', 'mov_5_pts', 'mov_5_poss', 'mov_5_pace',
       'mov_5_oeff', 'mov_5_deff', 'mov_5_bl', 'mov_5_away_1q',
       'mov_5_away_2q', 'mov_5_away_3q', 'mov_5_away_4q', 'mov_5_away_o

#### No Correlation between existing game data and covering

There is no correlation between any of the moving average data points and covering the spread.  We would expect strong correlations as these are both numbers available to the bookmakers and they are in isolation and do not take into account their opponents.  We will use these variables to engineer new features that hopefully will be useful in predicting whether or not a team covers. 

In [82]:
nba.corr()

,index,spread,total,moneyline,index.1,away_line,away_total,away_moneyline,mov_5_1q,mov_5_2q,...,mov_5_away_total_reb,mov_5_away_assists,mov_5_away_fouls,mov_5_away_steals,mov_5_away_turnovers,mov_5_away_blocks,mov_5_away_off_eff,mov_5_away_def_eff,line_cv,cover
index,1.000000,0.026392,0.068830,0.044917,1.000000,-0.026392,0.068830,-0.063135,0.042094,0.020738,...,0.025414,0.013818,-0.024198,0.023512,-0.017668,0.011105,0.018740,-0.065169,0.025218,0.025918
spread,0.026392,1.000000,0.033764,0.761264,0.026392,-1.000000,0.033764,-0.714668,-0.160330,-0.128406,...,0.115192,0.219505,-0.084720,0.117088,-0.168001,0.134671,0.281821,-0.297177,-0.006435,0.012572
total,0.068830,0.033764,1.000000,0.048448,0.068830,-0.033764,1.000000,-0.057921,0.308846,0.315637,...,0.072464,0.100725,0.072723,0.039230,0.057540,0.088246,0.114011,0.184834,0.008488,0.009268
moneyline,0.044917,0.761264,0.048448,1.000000,0.044917,-0.761264,0.048448,-0.977401,-0.107474,-0.088900,...,0.101537,0.188634,-0.046683,0.113487,-0.136660,0.098014,0.229570,-0.222614,0.026409,0.021430
index.1,1.000000,0.026392,0.068830,0.044917,1.000000,-0.026392,0.068830,-0.063135,0.042094,0.020738,...,0.025414,0.013818,-0.024198,0.023512,-0.017668,0.011105,0.018740,-0.065169,0.025218,0.025918
away_line,-0.026392,-1.000000,-0.033764,-0.761264,-0.026392,1.000000,-0.033764,0.714668,0.160330,0.128406,...,-0.115192,-0.219505,0.084720,-0.117088,0.168001,-0.134671,-0.281821,0.297177,0.006435,-0.012572
away_total,0.068830,0.033764,1.000000,0.048448,0.068830,-0.033764,1.000000,-0.057921,0.308846,0.315637,...,0.072464,0.100725,0.072723,0.039230,0.057540,0.088246,0.114011,0.184834,0.008488,0.009268
away_moneyline,-0.063135,-0.714668,-0.057921,-0.977401,-0.063135,0.714668,-0.057921,1.000000,0.090033,0.073133,...,-0.098647,-0.177374,0.049894,-0.111108,0.135258,-0.096234,-0.218228,0.209144,-0.024072,-0.016337
mov_5_1q,0.042094,-0.160330,0.308846,-0.107474,0.042094,0.160330,0.308846,0.090033,1.000000,0.299851,...,-0.015302,-0.011470,-0.020966,-0.053931,-0.029136,-0.001289,0.012713,0.057901,0.031392,0.008528
mov_5_2q,0.020738,-0.128406,0.315637,-0.088900,0.020738,0.128406,0.315637,0.073133,0.299851,1.000000,...,-0.025288,-0.058083,-0.029105,-0.042415,-0.013969,-0.028734,-0.001390,0.055701,0.016668,0.012961


In [83]:
#eff ratio ratio by line
nba['eff_ratio'] = (nba.mov_5_oeff/ nba.mov_5_away_def_eff) 
nba['eff_ratio1'] = (nba.mov_5_deff/ nba.mov_5_away_off_eff) 
nba['eff_ratio2'] = (nba.mov_5_oeff/ nba.mov_5_away_off_eff) 
nba['eff_ratio3'] = (nba.mov_5_deff/ nba.mov_5_away_off_eff) 

In [84]:
nba.cover.corr(nba.eff_ratio)

0.04628122610636807

In [85]:
nba.cover.corr(nba.eff_ratio3)

-0.030623539682131117

In [86]:
features = ['cover', 'line_cv', 'eff_ratio', 'eff_ratio1', 
            'eff_ratio2','eff_ratio3' ]

In [87]:
nba = nba[features]

In [88]:
nba.to_csv('./data/nba_analysis_data.csv', index = False)